In [1]:
import socket
import torch
from transformers import BertForSequenceClassification
import pickle

# Set up the server to communicate with the second part of the model
def start_server(host='0.0.0.0', port=1700):
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind((host, port))
    server_socket.listen(1)  # Listen for 1 connection
    print(f"Server is listening on {host}:{port}...")

    conn, addr = server_socket.accept()  # Accept the connection
    print(f"Connected by {addr}")

    return conn, server_socket

# Initialize tokenizer and dataset
model = BertForSequenceClassification.from_pretrained("google/bert_uncased_L-2_H-128_A-2", num_labels=2)

# Manually split the model: First part of the model (BERT encoder)
class BertPart1(torch.nn.Module):
    def __init__(self, model):
        super(BertPart1, self).__init__()
        self.bert = model.bert  # Extract the BERT part of the model

    def forward(self, input_ids, attention_mask):
        return self.bert(input_ids=input_ids, attention_mask=attention_mask)[0]  # Return hidden states

# Create the first part of the model
model_part1 = BertPart1(model)

# Use GPU if available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model_part1.to(device)

# Optimizer
optimizer = torch.optim.AdamW(model_part1.parameters(), lr=5e-5)

# Start the server
conn, server_socket = start_server()

# Assume input_ids and attention_mask are pre-defined tensors for demonstration
# Replace this with your actual DataLoader or batch processing
input_ids = torch.randint(0, 1000, (16, 256)).to(device)  # Random input for example
attention_mask = torch.ones(input_ids.shape, device=device)

# Forward pass and send data to the client
for _ in range(10):  # Example loop, replace with actual training loop
    optimizer.zero_grad()
    hidden_states = model_part1(input_ids, attention_mask)

    # Send the output to the client using a socket
    data_to_send = pickle.dumps(hidden_states.cpu().detach().numpy())  # Serialize the output
    conn.sendall(data_to_send)  # Send to client
    print("Sent hidden states to client")

# Close the connection and server socket
conn.close()
server_socket.close()

C:\Users\Exam\.conda\envs\NM21AI059\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Server is listening on 0.0.0.0:1700...
Connected by ('172.16.19.49', 49882)
Sent hidden states to client
Sent hidden states to client
Sent hidden states to client
Sent hidden states to client
Sent hidden states to client
Sent hidden states to client
Sent hidden states to client
Sent hidden states to client
Sent hidden states to client
Sent hidden states to client


In [2]:
print(model_part1)

BertPart1(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affin

In [3]:
import socket
import torch
from transformers import BertForSequenceClassification, BertTokenizer
import pickle

# Set up the server to communicate with the second part of the model
def start_server(host='0.0.0.0', port=1700):
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind((host, port))
    server_socket.listen(1)  # Listen for 1 connection
    print(f"Server is listening on {host}:{port}...")

    conn, addr = server_socket.accept()  # Accept the connection
    print(f"Connected by {addr}")

    return conn, server_socket

# Initialize tokenizer and dataset
tokenizer = BertTokenizer.from_pretrained("huawei-noah/TinyBERT_General_4L_312D")
model = BertForSequenceClassification.from_pretrained("huawei-noah/TinyBERT_General_4L_312D", num_labels=2)

# Manually split the model: First half (encoder)
class BertPart1(torch.nn.Module):
    def __init__(self, model):
        super(BertPart1, self).__init__()
        self.bert = model.bert  # Extract the BERT part of the model

    def forward(self, input_ids, attention_mask):
        return self.bert(input_ids=input_ids, attention_mask=attention_mask)[0]  # Return hidden states

# Create the first part of the model
model_part1 = BertPart1(model)

# Use GPU if available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model_part1.to(device)

# Optimizer
optimizer = torch.optim.AdamW(model_part1.parameters(), lr=5e-5)

# Start the server
conn, server_socket = start_server()

# Example data: Replace with your DataLoader or batch processing
# Assume input_ids and attention_mask are pre-defined tensors
input_ids = torch.randint(0, 1000, (16, 128)).to(device)  # Random input for example
attention_mask = torch.ones(input_ids.shape, device=device)

# Forward pass and send data to the client
for _ in range(10):  # Example loop, replace with actual training loop
    optimizer.zero_grad()
    hidden_states = model_part1(input_ids, attention_mask)

    # Send the output to the client using a socket
    data_to_send = pickle.dumps(hidden_states.cpu().detach().numpy())  # Serialize the output
    conn.sendall(data_to_send)  # Send to client
    print("Sent hidden states to client")

# Close the connection and server socket
conn.close()
server_socket.close()


C:\Users\Exam\.conda\envs\NM21AI059\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Exam\.cache\huggingface\hub\models--huawei-noah--TinyBERT_General_4L_312D. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at h

Server is listening on 0.0.0.0:1700...
Connected by ('172.16.19.49', 49995)
Sent hidden states to client
Sent hidden states to client
Sent hidden states to client
Sent hidden states to client
Sent hidden states to client
Sent hidden states to client
Sent hidden states to client
Sent hidden states to client
Sent hidden states to client
Sent hidden states to client


In [4]:
print(model_part1)

BertPart1(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 312, padding_idx=0)
      (position_embeddings): Embedding(512, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-3): 4 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affin

In [5]:
import socket
import torch
from transformers import BertForSequenceClassification, BertTokenizer
import pickle
from torch.utils.data import DataLoader, Dataset

# Set up the server to communicate with the second part of the model
def start_server(host='0.0.0.0', port=1700):
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind((host, port))
    server_socket.listen(1)  # Listen for 1 connection
    print(f"Server is listening on {host}:{port}...")

    conn, addr = server_socket.accept()  # Accept the connection
    print(f"Connected by {addr}")

    return conn, server_socket

# Custom Dataset for IMDb
class IMDBDataset(Dataset):
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        self.sentences = [
            "This movie was great!", "I did not enjoy this film.", 
            "It was an average movie.", "The plot was fantastic!", 
            "I wouldn't recommend it.", "It was a waste of time."
        ]
        self.labels = [1, 0, 0, 1, 0, 0]  # Sample labels

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        encoding = self.tokenizer.encode_plus(
            self.sentences[idx],
            add_special_tokens=True,
            max_length=128,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(self.labels[idx], dtype=torch.long)
        }

# Initialize tokenizer and model
tokenizer = BertTokenizer.from_pretrained("huawei-noah/TinyBERT_General_4L_312D")
model = BertForSequenceClassification.from_pretrained("huawei-noah/TinyBERT_General_4L_312D", num_labels=2)

# Manually split the model: Encoder and pooler
class BertPart1(torch.nn.Module):
    def __init__(self, model):
        super(BertPart1, self).__init__()
        self.bert = model.bert  # Extract the BERT part of the model

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        return outputs.last_hidden_state, outputs.pooler_output  # Return hidden states and pooled output

# Create the first part of the model
model_part1 = BertPart1(model)

# Use GPU if available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model_part1.to(device)

# Start the server
conn, server_socket = start_server()

# Create a DataLoader for the IMDb dataset
dataset = IMDBDataset(tokenizer)
data_loader = DataLoader(dataset, batch_size=2, shuffle=True)

# Forward pass and send data to the client
for batch in data_loader:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)

    hidden_states, pooled_output = model_part1(input_ids, attention_mask)

    # Send the output to the client using a socket
    data_to_send = {
        'hidden_states': hidden_states.cpu().detach().numpy(), 
        'pooled_output': pooled_output.cpu().detach().numpy()
    }
    conn.sendall(pickle.dumps(data_to_send))  # Send to client
    print("Sent hidden states and pooled output to client")

# Close the connection and server socket
conn.close()
server_socket.close()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Server is listening on 0.0.0.0:1700...
Connected by ('172.16.19.49', 50079)
Sent hidden states and pooled output to client
Sent hidden states and pooled output to client
Sent hidden states and pooled output to client


In [6]:
print(model_part1)

BertPart1(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 312, padding_idx=0)
      (position_embeddings): Embedding(512, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-3): 4 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affin

In [ ]:
import socket
import torch
from transformers import BertTokenizer, BertModel
import pickle
from torch.utils.data import DataLoader, Dataset
from datasets import load_dataset

# Set up the server to communicate with the second part of the model
def start_server(host='0.0.0.0', port=1700):
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind((host, port))
    server_socket.listen(1)  # Listen for 1 connection
    print(f"Server is listening on {host}:{port}...")

    conn, addr = server_socket.accept()  # Accept the connection
    print(f"Connected by {addr}")

    return conn, server_socket

# Custom Dataset for IMDb
class IMDBDataset(Dataset):
    def __init__(self, tokenizer, split):
        self.tokenizer = tokenizer
        self.data = load_dataset('imdb', split=split)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        encoding = self.tokenizer.encode_plus(
            item['text'],
            add_special_tokens=True,
            max_length=128,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        label = 1 if item['label'] == 'pos' else 0
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

# Initialize tokenizer and model
tokenizer = BertTokenizer.from_pretrained("huawei-noah/TinyBERT_General_4L_312D")
model = BertModel.from_pretrained("huawei-noah/TinyBERT_General_4L_312D")  # Only the encoder

# Manually split the model: Encoder only
class BertPart1(torch.nn.Module):
    def __init__(self, model):
        super(BertPart1, self).__init__()
        self.bert = model  # Use only the BERT part of the model

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        return outputs.last_hidden_state  # Return hidden states only

# Create the first part of the model
model_part1 = BertPart1(model)

# Use GPU if available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model_part1.to(device)

# Start the server
conn, server_socket = start_server()

# Create a DataLoader for the IMDb dataset
dataset = IMDBDataset(tokenizer, split='train')
data_loader = DataLoader(dataset, batch_size=2, shuffle=True)

# Forward pass and send data to the client
for batch in data_loader:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)

    hidden_states = model_part1(input_ids, attention_mask)

    # Send the output to the client using a socket
    conn.sendall(pickle.dumps(hidden_states.cpu().detach().numpy()))  # Send to client
    print("Sent hidden states to client")

# Close the connection and server socket
conn.close()
server_socket.close()


Server is listening on 0.0.0.0:1700...
Connected by ('172.16.19.49', 50106)
Sent hidden states to client
Sent hidden states to client
Sent hidden states to client
Sent hidden states to client
Sent hidden states to client
Sent hidden states to client
Sent hidden states to client
Sent hidden states to client
Sent hidden states to client
Sent hidden states to client
